In [1]:
magma_dir = '/home/ubuntu/magma/'
bucket_dir = '/home/ubuntu/s3/'
transformers_dir = '/home/ubuntu/transformers/'
cache_dir = bucket_dir+'.cache/'

## **Fine-tuning**

In [2]:
finetune_script = '"'+transformers_dir+'examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+transformers_dir+'examples/seq2seq/run_eval.py"'

### **Config**

In [3]:
import sys
sys.path.insert(0, magma_dir)
import config

import wandb
wandb.login()

project_name = 'hps_bart_para_wordembed_freezeembeds'
%env WANDB_PROJECT=$project_name

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: marcoabrate (use `wandb login --relogin` to force relogin)


env: WANDB_PROJECT=hps_bart_para_wordembed_freezeembeds


### HP search

In [4]:
model_name_or_path = 'sshleifer/distilbart-cnn-12-6'
model_name_or_path = 'facebook/bart-large-cnn'

In [5]:
data_dir = '"'+bucket_dir+'datasets/karger_books_para_wordembed/bart/st/"'

output_dir = '"'+bucket_dir+'fine-tuning/'+project_name+'"'

log_dir = output_dir + '/logs'

In [6]:
from transformers import AutoConfig
model_config = AutoConfig.from_pretrained(model_name_or_path, use_cache=False)
model_config.min_length = config.ONE_BULLET_MIN_LEN
model_config.max_length = config.ONE_BULLET_MAX_LEN
model_config.num_beams = 2

model_config.task_specific_params['summarization']['min_length'] = config.ONE_BULLET_MIN_LEN
model_config.task_specific_params['summarization']['max_length'] = config.ONE_BULLET_MAX_LEN
model_config.task_specific_params['summarization']['num_beams'] = 2
model_config_dir = '"'+bucket_dir+'fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_config"'
model_config.save_pretrained(model_config_dir[1:-1])

In [ ]:
!python3 $finetune_script \
--model_name_or_path $model_name_or_path \
--config_name $model_config_dir \
--tokenizer_name $model_name_or_path \
--cache_dir $cache_dir \
--data_dir $data_dir \
--freeze_embeds \
--do_train \
--learning_rate 5e-5 \
--label_smoothing_factor 0.1 \
--warmup_steps 0 \
--fp16 \
--sortish_sampler \
--task summarization \
--max_source_length 1024 \
--max_target_length $config.ONE_BULLET_MAX_LEN \
--val_max_target_length $config.ONE_BULLET_MAX_LEN \
--num_train_epochs 10 \
--logging_steps 20 --logging_first_step \
--per_device_train_batch_size 2 --per_device_eval_batch_size 8 \
--gradient_accumulation_steps 16 \
--evaluation_strategy steps --eval_steps 14 --eval_beams 2 \
--predict_with_generate \
--save_steps 5000 \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED \
--run_name $output_dir

02/15/2021 14:47:43 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: True
02/15/2021 14:47:43 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='/home/ubuntu/s3/fine-tuning/hps_bart_para_wordembed_freezeembeds', overwrite_output_dir=True, do_train=True, do_eval=None, do_predict=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=16, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_steps=0, logging_dir='runs/Feb15_14-47-43_ip-172-31-39-35', logging_first_step=True, logging_steps=20, save_steps=5000, save_total_

Downloading: 100%|█████████████████████████| 1.63G/1.63G [01:30<00:00, 17.9MB/s]
[INFO|file_utils.py:1302] 2021-02-15 14:49:17,259 >> storing https://huggingface.co/facebook/bart-large-cnn/resolve/main/pytorch_model.bin in cache at /home/ubuntu/s3/.cache/4ccdf4cdc01b790f9f9c636c7695b5d443180e8dbd0cbe49e07aa918dda1cef0.fa29468c10a34ef7f6cfceba3b174d3ccc95f8d755c3ca1b829aff41cc92a300
[INFO|file_utils.py:1305] 2021-02-15 14:49:37,027 >> creating metadata file for /home/ubuntu/s3/.cache/4ccdf4cdc01b790f9f9c636c7695b5d443180e8dbd0cbe49e07aa918dda1cef0.fa29468c10a34ef7f6cfceba3b174d3ccc95f8d755c3ca1b829aff41cc92a300
02/15/2021 14:49:37 - INFO - filelock -   Lock 140342032550936 released on /home/ubuntu/s3/.cache/4ccdf4cdc01b790f9f9c636c7695b5d443180e8dbd0cbe49e07aa918dda1cef0.fa29468c10a34ef7f6cfceba3b174d3ccc95f8d755c3ca1b829aff41cc92a300.lock
[INFO|modeling_utils.py:1027] 2021-02-15 14:49:37,156 >> loading weights file https://huggingface.co/facebook/bart-large-cnn/resolve/main/pytorch_mod

Batches: 100%|██████████| 1/1 [00:00<00:00, 139.12it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 141.75it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 96.07it/s]


...

02/15/2021 15:25:10 - INFO - gensim.models.base_any2vec -   training model with 3 workers on 2517 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5
02/15/2021 15:25:10 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 2 more threads
02/15/2021 15:25:10 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 1 more threads
02/15/2021 15:25:10 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 0 more threads
02/15/2021 15:25:10 - INFO - gensim.models.base_any2vec -   EPOCH - 1 : training on 11405 raw words (8735 effective words) took 0.0s, 208761 effective words/s
02/15/2021 15:25:10 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 2 more threads
02/15/2021 15:25:10 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 1 more threads
02/15/2021 15:25:10 - INFO - gensim.models.base_any2vec -   worker thread f

Batches: 100%|██████████| 1/1 [00:00<00:00, 136.67it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 135.76it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 140.96it/s]


Batches: 100%|██████████| 1/1 [00:00<00:00, 142.06it/s]
02/15/2021 15:30:32 - INFO - gensim.models.word2vec -   collecting all words and their counts
02/15/2021 15:30:32 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
02/15/2021 15:30:32 - INFO - gensim.models.word2vec -   collected 2574 word types from a corpus of 11202 raw words and 532 sentences
02/15/2021 15:30:32 - INFO - gensim.models.word2vec -   Loading a fresh vocabulary
02/15/2021 15:30:32 - INFO - gensim.models.word2vec -   effective_min_count=1 retains 2574 unique words (100% of original 2574, drops 0)
02/15/2021 15:30:32 - INFO - gensim.models.word2vec -   effective_min_count=1 leaves 11202 word corpus (100% of original 11202, drops 0)
02/15/2021 15:30:32 - INFO - gensim.models.word2vec -   deleting the raw counts dictionary of 2574 items
02/15/2021 15:30:32 - INFO - gensim.models.word2vec -   sample=0.001 downsamples 35 most-common words
02/15/2021 15:30:32 - INFO - ge